# Evaluacion Crediticia Preliminar con Perceptron
### Estructura de Datos y Algoritmos (T12A)

### Integrantes

* Castañeda De La Cruz, Edgar Daniel
* Jairo Villacorta

## Presentación

El presente documento es un informe tecnico de la implementacion de un software para ser utilizado como auxiliar en
la toma de deciciones para el area de *Riesgo Crediticio* de la empresa **CREDITOS E INVERSIONES LIMA E.I.R.L.**, 
registrado en sunat con numero de RUC **20407942800** esta pyme cuenta con una gran demora al momento de realizar los calculos de riesgos crediticios (entre 2 y 3 semanas), todo el trabajo es realizado manualmente y se basa principalmente en numerosos datos historicos de prestamos ya evaluados.

## Objetivos del Trabajo

* Minimizar drasticamente el tiempo de respuesta de evaluación crediticia.
* Desarrollar la solucion personalizada en base a su data historica.
* Nuestra solucion debe considerar que los datos historicos siempre estan en constante actualizacion.
* Asegurar la mantenibilidad del software a futuro.

## Descripción de la Empresa

### Organigrama
![organigrama](https://xmindshare.s3.amazonaws.com/preview/microfinance-organizational-chart-cmfnc-1335362344947.jpg)


### Proceso Actual de Evaluación

Antes de desembolsar un préstamo, la entidad siempre realizan una evaluación crediticia que dependerá de cada sucursal, pero todas siempre revisan que el solicitante no tenga deudas morosas en el sistema financiero, historial de pagos, y su comportamiento crediticio.

1. Si es un cliente que cumple con sus compromisos de pago en fecha. 
2. Cuál es la actividad económica (sector) a que se dedica y la antigüedad que tiene su negocio. 
3. Se tomará en cuenta su capacidad de pago, que es la cantidad de deuda que puede manejar de acuerdo a sus ingresos, los gastos del negocio y sus gastos personales (incluyendo las personas que dependan de él). 
4. En algunos casos se tendrá en cuenta si dispone de capital adicional, si tiene cuentas de ahorro o alguna propiedad que pueda funcionar como garantía del préstamo. 



## Descripcion de la Solución

Desarrollaremos un algoritmo capaz de aprender los patrones de datos recogidos en el archivo historico (previamente clasificado) de la empresa, con este aprendizaje sera posible responder con un Si o No a la pregunta *¿Existe un alto riesgo de que no pueda cumplir con el prestamo?* para lo cual modelaremos una neurona básica (perceptron) con funcion de activación binaria la cual describiremos a continuación:

### Perceptron
Algunas ideas simples de que es este algoritmo:

 * Primer modelo de red neuronal artificial desarrollado por Rosemblat en 1958. Consta de un numero arbitrario de capas.
 * Es un modelo unidireccional, compuesto por dos capas de neuronas, una sensorial o de entrada y otra de salida. La operación de una red de este tipo con "n" neuronas de entrada (Unidades de Procesamiento (U.P) y "m" de salida y su modo de entrenamiento es de tipo supervisado.
 * Determinar los pesos Sinápticos para que el dispositivo represente la relación entrada-salida lo mas fidedigna posible. La regla de aprendizaje permite corregir los pesos sinápticos para que alcancen los valore deseado, partiendo de un conjunto de pesos inicializados aleatoriamente


## Fases de Implementación

### F. Configuracion

Definicion de Hiperparametros y Carga de Datos en Memoria.

#### HIPERPARAMETROS

In [11]:
@eta = 0.00001 # TASA DE CONVERGENCIA (SUAVIZADO) 
@epochs = 100

100

#### CARGA DE DATOS A MEMORIA EN LISTA DE AMBITO GLOBAL @dataset

In [12]:
require 'csv'

@dataset = []

def cargar_data(file)
  # OBTENER CANTIDAD DE INPUTS
  i = 0
  
  CSV.foreach("#{file}.csv", :headers => true, converters: :numeric) do |row|
    @dataset[i] = row.fields
    i += 1
  end
  @ins = @dataset[i-1].size # CANTIDAD DE INPUTS
end

cargar_data('data')

8

#### GENERAR LISTA DE PESOS ALEATORIOS POR CADA INPUT (COLUMNA DE DATASET)

In [13]:
@w = []

i = 0
while i < @ins 
  @w[i] = rand(0..(@ins-1))/10.00
  i += 1
end

@w


[0.4, 0.0, 0.7, 0.6, 0.6, 0.1, 0.3, 0.2]

#### DEFINIR LA FUNCION DE ACTIVACION TIPO sigmoidal binaria 

In [14]:
def sigmoidal(inputs)
  i = 0
  weighted_average = 0
  inputs = [1] + inputs
  # OBTENER SUMA DEL PRODUCTO DE LOS PESOS
  while i < @ins
    wn = @w[i]
    xn = inputs[i]
    weighted_average += wn*xn
    i += 1
  end
  return weighted_average
end

def predict(inputs)
  weighted_average = sigmoidal(inputs)
  if weighted_average > 0
    return 1  
  else
    return 0
  end
end

:predict

#### DEFINIR FUNCION DE ENTRENAMIENTO

In [15]:
def train(inputs, ex_output)
  output = predict(inputs)
  inputs = [1] + inputs
  error = ex_output - output
  if error != 0
      # SI EXISTE EL ERROR REAJUSTAR LOS PESOS
      i = 0
      while i < @ins
        wn = @w[i]
        xn = inputs[i]
        @w[i] = wn + (@eta*error*xn)
        i += 1
      end
  end
  return error
end              

:train

### F. Entrenamiento

In [16]:
weights = [] # Lista con los pesos
errors = []  # Lista con los errores

data_x = []
data_y = []
i = 0
while i <= @epochs
  # ENTRENAREMOS VARIAS VECES SOBRE EL MISMO DATASET
  # PARA QUE LOS PESOS CONVERGAN
  sum_errors = 0
  @dataset.each_with_index do | person, index |
    # OBTENER EL ULTIMO ELEMENTO DE LA LISTA
    output = person[-1]
    inp = person.take person.size - 1
    weights.push(@w)
    err = train(inp, output)
    errors.push(err)
    sum_errors += err
  end
  # GUARDAR DATOS PARA LA REPRESENTACION
  data_y.push(@ins - sum_errors.round(2))
  data_x.push(i)
  i += 1
end


#### REPRESENTACION DE LA PRECICION EN CADA ITERACION

In [17]:
require 'nyaplot'

plot = Nyaplot::Plot.new
line_pres = plot.add(:line, data_x, data_y)
line_pres.color('#ff2680')

plot.x_label("Iteraciones")
plot.y_label("Errores")
plot.show

#<Nyaplot::Frame:0x000000014f03c0 @properties={:panes=>[#<Nyaplot::Plot:0x0000000150c390 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000000014f0b40 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"#ff2680"}, :data=>"bed94897-bcf1-4aae-9131-4c17c6d2e177"}, @xrange=[0, 100], @yrange=[9.0, 68.0]>], :options=>{:x_label=>"Iteraciones", :y_label=>"Errores", :zoom=>true, :width=>700, :xrange=>[0, 100], :yrange=>[9.0, 68.0]}}>], :data=>{"bed94897-bcf1-4aae-9131-4c17c6d2e177"=>#<Nyaplot::DataFrame:0x00000001507bb0 @name="bed94897-bcf1-4aae-9131-4c17c6d2e177", @rows=[{:data0=>0, :data1=>68.0}, {:data0=>1, :data1=>25.0}, {:data0=>2, :data1=>22.0}, {:data0=>3, :data1=>21.0}, {:data0=>4, :data1=>21.0}, {:data0=>5, :data1=>21.0}, {:data0=>6, :data1=>20.0}, {:data0=>7, :data1=>20.0}, {:data0=>8, :data1=>17.0}, {:data0=>9, :data1=>19.0}, {:data0=>10, :data1=>18.0}, {:data0=>11, :data1=>16.0}, {:data0=>12, :data1=>17.0}, {:data0=>13, :data1=>17.0}, {:data0=>14, :data1=>16.0}, {:data0=>15, :data1=>16.0}, {:data0=>16, :data1=>15.0}, {:data0=>17, :data1=>15.0}, {:data0=>18, :data1=>15.0}, {:data0=>19, :data1=>15.0}, {:data0=>20, :data1=>15.0}, {:data0=>21, :data1=>15.0}, {:data0=>22, :data1=>15.0}, {:data0=>23, :data1=>15.0}, {:data0=>24, :data1=>15.0}, {:data0=>25, :data1=>15.0}, {:data0=>26, :data1=>15.0}, {:data0=>27, :data1=>14.0}, {:data0=>28, :data1=>12.0}, {:data0=>29, :data1=>12.0}, {:data0=>30, :data1=>12.0}, {:data0=>31, :data1=>12.0}, {:data0=>32, :data1=>12.0}, {:data0=>33, :data1=>11.0}, {:data0=>34, :data1=>12.0}, {:data0=>35, :data1=>11.0}, {:data0=>36, :data1=>12.0}, {:data0=>37, :data1=>11.0}, {:data0=>38, :data1=>12.0}, {:data0=>39, :data1=>10.0}, {:data0=>40, :data1=>11.0}, {:data0=>41, :data1=>11.0}, {:data0=>42, :data1=>10.0}, {:data0=>43, :data1=>11.0}, {:data0=>44, :data1=>11.0}, {:data0=>45, :data1=>10.0}, {:data0=>46, :data1=>10.0}, {:data0=>47, :data1=>11.0}, {:data0=>48, :data1=>10.0}, {:data0=>49, :data1=>11.0}, {:data0=>50, :data1=>10.0}, {:data0=>51, :data1=>10.0}, {:data0=>52, :data1=>9.0}, {:data0=>53, :data1=>11.0}, {:data0=>54, :data1=>10.0}, {:data0=>55, :data1=>10.0}, {:data0=>56, :data1=>9.0}, {:data0=>57, :data1=>10.0}, {:data0=>58, :data1=>10.0}, {:data0=>59, :data1=>10.0}, {:data0=>60, :data1=>9.0}, {:data0=>61, :data1=>9.0}, {:data0=>62, :data1=>10.0}, {:data0=>63, :data1=>10.0}, {:data0=>64, :data1=>10.0}, {:data0=>65, :data1=>9.0}, {:data0=>66, :data1=>9.0}, {:data0=>67, :data1=>10.0}, {:data0=>68, :data1=>10.0}, {:data0=>69, :data1=>9.0}, {:data0=>70, :data1=>9.0}, {:data0=>71, :data1=>9.0}, {:data0=>72, :data1=>9.0}, {:data0=>73, :data1=>10.0}, {:data0=>74, :data1=>10.0}, {:data0=>75, :data1=>9.0}, {:data0=>76, :data1=>9.0}, {:data0=>77, :data1=>9.0}, {:data0=>78, :data1=>9.0}, {:data0=>79, :data1=>9.0}, {:data0=>80, :data1=>9.0}, {:data0=>81, :data1=>10.0}, {:data0=>82, :data1=>9.0}, {:data0=>83, :data1=>9.0}, {:data0=>84, :data1=>9.0}, {:data0=>85, :data1=>9.0}, {:data0=>86, :data1=>9.0}, {:data0=>87, :data1=>9.0}, {:data0=>88, :data1=>10.0}, {:data0=>89, :data1=>9.0}, {:data0=>90, :data1=>9.0}, {:data0=>91, :data1=>9.0}, {:data0=>92, :data1=>9.0}, {:data0=>93, :data1=>9.0}, {:data0=>94, :data1=>9.0}, {:data0=>95, :data1=>9.0}, {:data0=>96, :data1=>10.0}, {:data0=>97, :data1=>9.0}, {:data0=>98, :data1=>9.0}, {:data0=>99, :data1=>9.0}, {:data0=>100, :data1=>9.0}]>}, :extension=>[]}>

#### REPRESENTACION DE LA DISTRIBUCION DEL PATRON

In [18]:
data_x = [[], []]
data_y = [[], []]

@dataset.each_with_index do | person, index |
  # OBTENER EL ULTIMO ELEMENTO DE LA LISTA
  tag = person[-1]
  inp = person.take person.size - 1
  data_x[tag].push(sigmoidal(inp))
  data_y[tag].push(index)
end

plot = Nyaplot::Plot.new
line_pres = plot.add(:scatter, data_x[0], data_y[0])
line_pres.color('#ff2680')

line_pres = plot.add(:scatter, data_x[1], data_y[1])
line_pres.color('#15db29')

plot.show

#<Nyaplot::Frame:0x00000000dd2430 @properties={:panes=>[#<Nyaplot::Plot:0x00000001131a90 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000001027d20 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"#ff2680"}, :data=>"fec987db-0ea2-46bf-93a3-24468ac52e76"}, @xrange=[-391.6757899999962, 2.0028800000017646], @yrange=[3, 844]>, #<Nyaplot::Diagram:0x00000000dd4ed8 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"#15db29"}, :data=>"e96e735b-fce6-4057-823b-7217d4895b25"}, @xrange=[5.332070000021827, 1452.7452100000264], @yrange=[0, 846]>], :options=>{:zoom=>true, :width=>700, :xrange=>[-391.6757899999962, 1452.7452100000264], :yrange=>[0, 846]}}>], :data=>{"fec987db-0ea2-46bf-93a3-24468ac52e76"=>#<Nyaplot::DataFrame:0x00000001131040 @name="fec987db-0ea2-46bf-93a3-24468ac52e76", @rows=[{:data0=>-307.5472399999955, :data1=>3}, {:data0=>-145.9321499999976, :data1=>4}, {:data0=>-30.565579999997908, :data1=>9}, {:data0=>-229.83960999999672, :data1=>10}, {:data0=>-221.18769999999722, :data1=>13}, {:data0=>-278.1876299999966, :data1=>14}, {:data0=>-106.61130999999735, :data1=>15}, {:data0=>-35.317119999998035, :data1=>16}, {:data0=>-153.9191399999977, :data1=>18}, {:data0=>-63.372859999997495, :data1=>23}, {:data0=>-156.49386999999706, :data1=>24}, {:data0=>-337.26097999999627, :data1=>26}, {:data0=>-86.05327999999827, :data1=>27}, {:data0=>-329.2293599999957, :data1=>28}, {:data0=>-32.19524999999851, :data1=>29}, {:data0=>-178.52617999999774, :data1=>30}, {:data0=>-112.0480599999974, :data1=>31}, {:data0=>-14.099819999997857, :data1=>33}, {:data0=>-240.50129999999626, :data1=>36}, {:data0=>-74.50410999999782, :data1=>37}, {:data0=>-291.1711499999958, :data1=>38}, {:data0=>-227.33915999999732, :data1=>39}, {:data0=>-312.2263199999958, :data1=>41}, {:data0=>-106.47279999999806, :data1=>42}, {:data0=>-54.17224999999839, :data1=>43}, {:data0=>-225.01984999999755, :data1=>46}, {:data0=>-178.33653999999814, :data1=>47}, {:data0=>-275.0868499999964, :data1=>49}, {:data0=>-83.23290999999803, :data1=>51}, {:data0=>-293.12904999999546, :data1=>52}, {:data0=>-241.46625999999682, :data1=>53}, {:data0=>-302.4627499999952, :data1=>54}, {:data0=>-173.8946099999977, :data1=>55}, {:data0=>-244.29309999999634, :data1=>59}, {:data0=>-186.87770999999648, :data1=>62}, {:data0=>-275.7868899999957, :data1=>68}, {:data0=>-210.7047899999977, :data1=>69}, {:data0=>-125.71551999999855, :data1=>73}, {:data0=>-198.65657999999698, :data1=>76}, {:data0=>-315.7040199999965, :data1=>77}, {:data0=>-248.5083799999969, :data1=>78}, {:data0=>-376.0408099999957, :data1=>80}, {:data0=>-92.82954999999846, :data1=>82}, {:data0=>-178.19349999999721, :data1=>83}, {:data0=>-138.4435899999975, :data1=>86}, {:data0=>-192.687689999997, :data1=>91}, {:data0=>-202.6146099999968, :data1=>92}, {:data0=>-322.61127999999655, :data1=>95}, {:data0=>-244.298829999996, :data1=>97}, {:data0=>-36.73576999999771, :data1=>98}, {:data0=>-66.90792999999915, :data1=>101}, {:data0=>-286.7169899999971, :data1=>102}, {:data0=>-366.17973999999595, :data1=>104}, {:data0=>-337.534799999996, :data1=>105}, {:data0=>-243.6341499999962, :data1=>106}, {:data0=>-292.68372999999616, :data1=>107}, {:data0=>-301.01039999999523, :data1=>108}, {:data0=>-254.40910999999588, :data1=>109}, {:data0=>-304.888209999996, :data1=>115}, {:data0=>-69.00068999999849, :data1=>117}, {:data0=>-218.13157999999768, :data1=>118}, {:data0=>-216.37431999999737, :data1=>121}, {:data0=>-160.17094999999705, :data1=>122}, {:data0=>-128.2795599999979, :data1=>123}, {:data0=>-92.4381199999983, :data1=>124}, {:data0=>-192.56662999999674, :data1=>126}, {:data0=>-346.77786999999574, :data1=>127}, {:data0=>-227.1246999999977, :data1=>129}, {:data0=>-226.6937199999977, :data1=>130}, {:data0=>-222.27387999999704, :data1=>132}, {:data0=>-199.671499999998, :data1=>134}, {:data0=>-165.91044999999667, :data1=>135}, {:data0=>-139.89323999999718, :data1=>137}, {:data0=>-321.68018999999634, :data1=>138

## F. De Pruebas y Producción

### Cotización #1
Cliente: EMPRESA TURISTICA SOL Y MAR E.I.R.L. | *RUC: 20140917690*

#### Evaluación
Cliente percibe un ingreso fijo mensual de **S/. 2100.00**, ademas de justificar otros ingresos por facturas de servicio por **S/. 1600.00** Aprox, tiene con nosotros **15 Letras** Pagadas, el importe de su ultima letra cobrada fue de **S/. 110.00**, Edad del cliente al solicitar nuevo prestamo es de **36 años**. Pertenece al **Sector 7** (PYMES De Servicios), Antiguedad del cliente **7 meses**.


In [19]:
ingreso_mensual = 2100
otros_ingresos = 1600
facturas = 15
importe_ultima_fact = 110
edad = 36
meses_contrato = 7
sector = 7

resp = predict([
  ingreso_mensual,
  otros_ingresos,
  facturas,
  importe_ultima_fact,
  edad,
  meses_contrato,
  sector
])

if resp == 1
  print "Asignar Prestamo\n"
else
  print "NO Asignar Prestamo\n"
end


Asignar Prestamo


### Cotización #2
Cliente: SERVICIOS GENERALES DE ALQUILER A & J E.I.R.L. | *RUC: 20529360712*

#### Evaluación
Cliente percibe un ingreso fijo mensual de **S/. 1350.00**, ademas de justificar otros ingresos por facturas de servicio por **S/. 360.00** Aprox, tiene con nosotros **10 Letras** Pagadas, el importe de su ultima letra cobrada fue de **S/. 150.00**, Edad del cliente al solicitar nuevo prestamo es de **33 años**. Pertenece al **Sector 2** (PYMES Alquiler y Arrendamiento), Antiguedad del cliente **23 meses**.


In [20]:
ingreso_mensual = 1350
otros_ingresos = 360
facturas = 10
importe_ultima_fact = 150
edad = 33
meses_contrato = 23
sector = 2


resp = predict([
  ingreso_mensual,
  otros_ingresos,
  facturas,
  importe_ultima_fact,
  edad,
  meses_contrato,
  sector
])

if resp == 1
  print "Asignar Prestamo\n"
else
  print "NO Asignar Prestamo\n"
end

NO Asignar Prestamo


## Conclusiones y Recomendaciones

Esta implementacion de la neurona tiene la capacidad de seguir aprendiendo con nuevos datos, pero tiene un limite en la curva de aprendizaje, por lo cual se recomienda mantener la informacion balanceada, y eliminar los datos mas antiguos y de uso poco frecuente, actualizar la trama de informacion antes de los dias de mayor actividad comercial.

## Referencias y bibliografía utilizada.

*Clever Algorithms: Nature-Inspired Programming Recipes | 
By Jason Brownlee PhD*